# Tabu Search

## Definition

The second meta-heuristic algorithm that will be implemented to the TSP is 'Tabu Search'. This algorithm consists of using an iterative solution implementation of a set of problem solutions and moving from one solution to another in the same neighbourhood of each related solution. This means maintaining a short term memory of specific changes of recent moves within the search space and preventing future moves from undoing those changes (Panigrahi, 2015).

Tabu Search moves towards a solution $s*$ that attempts to minimise $f$, a cost function. A class of simple modifications is defined that is able to be applied to a solution ready to advance to the next one. The solution $s'$ is obtained by modifying $m$ to solution $s$. The set of the acceptable modifications we call $M$ which is displayed as 

$$s' = s \oplus m, m \in M$$

this actuates the neighbourhood definition $N(s)$ of every solution $s$.

The main feature of Tabu Search is to forbid any moves that could bring the process back to one of the previously visited solutions. The final moves can be collected in a list $T$ of all forbidden moves. For a fixed number of iterations, a move will remain forbidden and the oldest element will be deleted on each iteration. The final modification will be added to the head of the queue.

The tabu status is assigned to different constituents $t_i$ of moves. Every constituent of the forbidden moves is collected in a separate list such as $T_i$ and tabu conditions of the form

$$t_i(m)\in T_i (i = 1, ... ,t)$$

are introduced. Move $m$ will then be forbidden if it satisfies all tabu conditions.


(C.-N. Fiechter, 1992)

## Code

In [ ]:
import math, time, random, sys
from random import randint, shuffle
import networkx as graphs
import pandas as pd
import numpy as np

class CompleteWGraph:
    n = 0
    p = 0
    lower_weight = 0
    upper_weight = 0
    distmatrix = {}
    w_edges = []
    def __init__(self,n,p,lower_weight,upper_weight):
        """n: number of nodes
        p: prob of 2 nodes being connected (between 0-1)
        lower/upper weight: range of possible weight values"""
        self.n = n
        self.p = p
        self.lower_weight= lower_weight
        self.upper_weight = upper_weight
    def random_weighted_graph(self):
        g = graphs.gnp_random_graph(self.n,self.p)
        m = g.number_of_edges()
        weights = [random.randint(self.lower_weight, self.upper_weight) for r in range(m)]
        #unweighted connections
        uw_edges = g.edges()
        # Create weighted graph edge list
        i=0
        w_edges = []
        ret_graph = graphs.Graph()
        for edge in uw_edges:
        #w_edges = [uw_edges[i][0], uw_edges[i][1], weights[i]]
        #w_edges+={(edge[0],edge[1]):weights[i]}
            ret_graph.add_edge(edge[0],edge[1],weight=weights[i])
            i =i +1
        #print(w_edges)
        #return graphs.Graph(w_edges, weighted = True,s=weights)
        return ret_graph

In [5]:
#g1_data = CompleteWGraph(10,0.6,5,20)
#g1 = g1_data.random_weighted_graph()

#weightdict = g1.get_edge_data(0,2)
#print(weightdict)
#print(g1.get_edge_data(0,2)['weight'])

#g1_data.createDistanceMatrix(g1)
#print("g1 distance matrix")
#print(g1_data.distmatrix)

###  Data Format is dict:
#           data[node_name] = gives you a list of link info
#           data[link_index][0] = name of node that edge goes to
#           data[link_index][1] = weight of that edge
def cook_data(nodes, probability, min_weight, max_weight):
    linkset = []
    links = {}

    if min_weight>max_weight:
        print('Lower weight cannot be greater then upper weight for the weight range. ')
        sys.exit()
    if probability<0 or probability>1:
        print('Probability incorrect. Must be between 0 and 1. ')
        sys.exit()
    g1_data = CompleteWGraph(nodes, probability, min_weight, max_weight)
    g1 = g1_data.random_weighted_graph()
    node_list=list(g1.nodes())
    max_weight_of_all=0
    for a in node_list:
        for b in node_list:
            if a==b:
                continue
            link = []
            link.append(a)
            link.append(b)
            edge_weight=g1.get_edge_data(a,b)['weight']
            link.append(edge_weight)
            linkset.append(link)
            print('%d %d %d' % (a,b,edge_weight))
            if edge_weight>max_weight_of_all:
                max_weight_of_all=edge_weight

    for link in linkset:
        try:
            linklist = links[str(link[0])]
            linklist.append(link[1:])
            links[str(link[0])] = linklist
        except:
            links[str(link[0])] = [link[1:]]

    return links, max_weight_of_all

In [3]:
def getNeighbors(state):
    #return hill_climbing(state)
    return two_opt_swap(state)

def hill_climbing(state):
    node = randint(1, len(state)-1)
    neighbors = []

    for i in range(len(state)):
        if i != node and i != 0:
            tmp_state = state.copy()
            tmp = tmp_state[i]
            tmp_state[i] = tmp_state[node]
            tmp_state[node] = tmp
            neighbors.append(tmp_state)

    return neighbors

def two_opt_swap(state):
    global neighborhood_size
    neighbors = []

    for i in range(neighborhood_size):
        node1 = 0
        node2 = 0

        while node1 == node2:
            node1 = randint(1, len(state)-1)
            node2 = randint(1, len(state)-1)

        if node1 > node2:
            swap = node1
            node1 = node2
            node2 = swap


        tmp = state[node1:node2]
        tmp_state = state[:node1] + tmp[::-1] +state[node2:]
        neighbors.append(tmp_state)

    return neighbors

def fitness(route, graph):
    path_length = 0

    for i in range(len(route)):
        if(i+1 != len(route)):
            dist = weight_distance(route[i], route[i+1], graph)
            if dist != -1:
                path_length = path_length + dist
            else:
                return max_fitness # there is no  such path

        else:
            dist = weight_distance(route[i], route[0], graph)
            if dist != -1:
                path_length = path_length + dist
            else:
                return max_fitness # there is no  such path

    return path_length

# not used in this code but some datasets has 2-or-more dimensional data points, in this case it is usable
def euclidean_distance(city1, city2):
    return math.sqrt((city1[0] - city2[0])**2 + ((city1[1] - city2[1])**2))

def weight_distance(city1, city2, graph):
    global max_fitness

    neighbors = graph[str(city1)]

    for neighbor in neighbors:
        if neighbor[0] == int(city2):
            return neighbor[1]

    return -1 #there can't be minus distance, so -1 means there is not any city found in graph or there is not such edge

In [ ]:
def tabu_search(nodes, probability, min_weight, max_weight):
    global max_fitness, start_node
    graph, max_weight = cook_data(nodes, probability, min_weight, max_weight)

    ## Below, get the keys (node names) and shuffle them, and make start_node as start
    s0 = list(graph.keys())
    shuffle(s0)

    if int(s0[0]) != start_node:
        for i in range(len(s0)):
            if  int(s0[i]) == start_node:
                swap = s0[0]
                s0[0] = s0[i]
                s0[i] = swap
                break;

    # max_fitness will act like infinite fitness
    max_fitness = ((max_weight) * (len(s0)))+1
    sBest = s0
    vBest = fitness(s0, graph)
    bestCandidate = s0
    tabuList = []
    tabuList.append(s0)
    stop = False
    best_keep_turn = 0

    start_time = time.time()
    while not stop :
        sNeighborhood = getNeighbors(bestCandidate)
        bestCandidate = sNeighborhood[0]
        for sCandidate in sNeighborhood:
            if (sCandidate not in tabuList) and ((fitness(sCandidate, graph) < fitness(bestCandidate, graph))):
                bestCandidate = sCandidate

        if (fitness(bestCandidate, graph) < fitness(sBest, graph)):
            sBest = bestCandidate
            vBest = fitness(sBest, graph)
            best_keep_turn = 0

        tabuList.append(bestCandidate)
        if (len(tabuList) > maxTabuSize):
            tabuList.pop(0)

        if best_keep_turn == stoppingTurn:
            stop = True

        best_keep_turn += 1

    exec_time =  time.time() - start_time
    return sBest, vBest, exec_time



## Tabu Search Takes edge-list in a given format:
#nodefrom nodeto weight
#0 1 5
#3 2 4
#1 0 3
#Undirectional edges should be written 2 times for both nodes.
maxTabuSize = 10000
neighborhood_size = 500
stoppingTurn = 500
max_fitness = 0
start_node = 0
solution, value, exec_time = tabu_search(nodes=100, probability=1, min_weight=10, max_weight=20)

print(solution)
print('----> '.join(a for a in solution))
print('Shortest Distance : %d' %(value))
print('Solved in %s seconds'%exec_time)


## References

* Ozan Polatbilek, (Jul 9, 2019) *Tabu search on Travelling Salesman Problem* [online] Available at: <https://github.com/polatbilek/Tabu-search-on-Travelling-Salesman-Problem/blob/master/tabu_search.py> [Accessed on Apr 8, 2020]

* C.-N. Fiechter, (July 16, 1992) *A parallel tabu search algorithm for large traveling salesman problems* [online] Available at: <https://core.ac.uk/download/pdf/82688573.pdf> [Accessed on Apr 12, 2020]